<a href="https://colab.research.google.com/github/Toramaru42/djl-smartgrid-project/blob/main/notebooks/01_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#モジュールimport
import pandas as pd
import numpy as np
import os

# グラフ確認のため
import matplotlib.pyplot as plt
import seaborn as sns
!pip install autoviz
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.3/170.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.8 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4
Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


In [6]:
#データの読み込み
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# データパスの指定（必要に応じて変更）
data_path = "/content/drive/MyDrive/卒論/data/electricity_marketing_dataset.csv"
df = pd.read_csv(data_path)

# 最初の5行を表示
df.head()

,timestamp,temperature,humidity,is_weekend,is_holiday,consumer_type,price_signal,historical_avg_demand,voltage_level,grid_frequency,energy_source_mix,demand_category
0,2024-01-01 00:00:00,0.526922,0.265203,0,1,residential,0.469818,0.080994,0.538437,0.228592,0.440540,Medium
1,2024-01-01 01:00:00,0.437412,0.539677,0,1,residential,0.431329,0.080842,0.412133,0.478937,0.312881,Medium
2,2024-01-01 02:00:00,0.548204,0.632031,0,1,commercial,0.712748,0.434790,0.784449,0.211724,0.127635,High
3,2024-01-01 03:00:00,0.671595,0.254353,0,1,commercial,0.708677,0.641761,0.596124,0.483553,0.468987,High
4,2024-01-01 04:00:00,0.423895,0.135163,0,1,residential,0.537836,0.585632,0.535468,0.396965,0.121962,Medium


In [9]:
df_av = AV.AutoViz(
    filename='',
    dfte=df,
    depVar='demand_category'  # 目的変数名（連続化前でOK）
)


Shape of your Data Set loaded: (720, 12)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  7
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  1
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  2
    Number of Discrete String Columns =  0
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  1
    Number of Columns to Delete =  0
    11 Predictors classified...
        1 variable(s) removed since they were ID or low-information variables
        List of variables removed: ['timestamp']

################ Multi_Classification problem ########

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
temperature,float64,0.000000,NA,0.000000,1.000000,Column has 5 outliers greater than upper bound (0.83) or lower than lower bound(0.08). Cap them or remove them.
humidity,float64,0.000000,NA,0.000000,1.000000,No issue
is_weekend,int64,0.000000,0,0.000000,1.000000,No issue
is_holiday,int64,0.000000,0,0.000000,1.000000,No issue
consumer_type,object,0.000000,0,,,No issue
price_signal,float64,0.000000,NA,0.000000,1.000000,No issue
historical_avg_demand,float64,0.000000,NA,0.000000,1.000000,No issue
voltage_level,float64,0.000000,NA,0.000000,1.000000,Column has 5 outliers greater than upper bound (0.97) or lower than lower bound(0.09). Cap them or remove them.
grid_frequency,float64,0.000000,NA,0.000000,1.000000,Column has 13 outliers greater than upper bound (0.82) or lower than lower bound(0.06). Cap them or remove them.
energy_source_mix,float64,0.000000,NA,0.000000,1.000000,No issue


Total Number of Scatter Plots = 28
All Plots done
Time to run AutoViz = 14 seconds 

 ###################### AUTO VISUALIZATION Completed ########################


In [11]:
#Yの整形
# 需要カテゴリを連続値にマップ
demand_map = {'Low': 0.0, 'Medium': 0.5, 'High': 1.0}
df['Y'] = df['demand_category'].map(demand_map)

# 確認
df[['demand_category', 'Y']].head()

,demand_category,Y
0,Medium,0.5
1,Medium,0.5
2,High,1.0
3,High,1.0
4,Medium,0.5


In [13]:
# カテゴリ変数をOne-Hotに
df_cat = pd.get_dummies(df['consumer_type'], prefix='consumer')

# 数値特徴量
df_num = df[['temperature', 'humidity', 'is_weekend', 'is_holiday',
             'historical_avg_demand', 'voltage_level',
             'grid_frequency', 'energy_source_mix']]

# 統合
X = pd.concat([df_num, df_cat], axis=1)
print("X shape:", X.shape)

X shape: (720, 11)


In [14]:
#Aを抽出
A = df[['price_signal']].values  # 2D arrayにしておくとあとで便利
print("A shape:", A.shape)

A shape: (720, 1)


In [15]:
# 保存フォルダを作成（必要なら）
os.makedirs("data", exist_ok=True)

# 保存
X.to_csv("data/X.csv", index=False)
pd.DataFrame(A, columns=["A"]).to_csv("data/A.csv", index=False)
df[['Y']].to_csv("data/Y.csv", index=False)